In [7]:
import pandas as pd
from urllib import request
import json
from pandas.io.json import json_normalize

url = 'https://api.production.mtomady.com/hack_hpi/claims?page=1'
headers = {}

r = request.Request(url, headers = headers)
response = request.urlopen(r)
elevations = response.read()
data = json.loads(elevations)

for i in range(1,data['meta']['total_pages']+1):
    url = 'https://api.production.mtomady.com/hack_hpi/claims?page={}'.format(i)
    headers = {}

    r = request.Request(url, headers = headers)
    response = request.urlopen(r)
    elevations = response.read()
    data = json.loads(elevations)
    
    df_1 = json_normalize(data['claims'])

    if i == 1:
        df_2 = df_1
    else:
        df_2 = pd.concat([df_2,df_1], ignore_index=True)

In [9]:
for index, item in enumerate(df_2.applied_treatments[df_2.applied_treatments.apply(lambda x:len(x)) > 0]):
    if index == 0:
        name_ind = list(df_2[df_2.applied_treatments.apply(lambda x:len(x)) > 0].loc[:,'health_care_provider.name'].index)[index]
        for i, d in enumerate(item):
            if i == 0:
                a = json_normalize(d)
                exploded_df_treat = a.join(pd.DataFrame([df_2[df_2.applied_treatments.apply(lambda x:len(x)) > 0].loc[name_ind,'health_care_provider.name']], columns = ['health_care_provider.name'], index = [0]))
            else:
                a = json_normalize(d)
                a = a.join(pd.DataFrame([df_2[df_2.applied_treatments.apply(lambda x:len(x)) > 0].loc[name_ind,'health_care_provider.name']], columns = ['health_care_provider.name']))
                exploded_df_treat = pd.concat([exploded_df_treat,a])

    else:
        name_ind = list(df_2[df_2.applied_treatments.apply(lambda x:len(x)) > 0].loc[:,'health_care_provider.name'].index)[index]
        for i, d in enumerate(item):
                a = json_normalize(d)
                a = a.join(pd.DataFrame([df_2[df_2.applied_treatments.apply(lambda x:len(x)) > 0].loc[name_ind,'health_care_provider.name']], columns = ['health_care_provider.name']))
                exploded_df_treat = pd.concat([exploded_df_treat,a])

exploded_df_treat['total_amount'] = pd.to_numeric( exploded_df_treat['total_amount'])


# Prediktor & Reminder

In [19]:
from twilio.rest import Client
def Send_to_Phone(msg):# put your twilio credentials here
    account_sid = no.
    auth_token = no.
    client = Client(account_sid, auth_token)

    response = client.messages.create(
                                to= no.,
                                from_= no.,
                                body= msg,
    )

In [20]:
import pandas as pd
import numpy as np
from urllib import request
import json
from pandas.io.json import json_normalize
from datetime import date
import time

def data_framer(list_o_url, headers):
    for url in list_o_url:
        headers = headers

        r = request.Request(url, headers = headers)
        response = request.urlopen(r)
        elevations = response.read()
        data = json.loads(elevations)
        
        df_1 = json_normalize(data['claims'])
        if i == 1:
            df = df_1
        else:
            df = pd.concat([df_2,df_1], ignore_index=True)
    return df
    
def last_treatment(df):
    df['submitted_at']=pd.to_datetime(df['submitted_at']) 
    df_last_treatment = df[df['demanded_amount']>1000].groupby('patient_id')['submitted_at'].max() #TODO Das muss genauer gehen
    return df_last_treatment
   
def msg_token_builder(treatment, timeframe):
    msg_token = "Veuillez prendre rendez-vous pour {} dans les {} prochaines semaines.".format(treatment, timeframe)
    return msg_token

def reminder(df): #urls, header):
#     df = data_framer(urls, header)
    
    today = date.today()
    patient_ID = last_treatment(df)[(today - pd.to_datetime(last_treatment(df)).apply(lambda x: x.date())).dt.days > 50].index
    msg_token = pd.DataFrame([msg_token_builder('échographie',3) for item in patient_ID], columns = ['msg_tokens'])
    
    for i in range(msg_token.shape[0]):
        time.sleep(0.5)
        if i == 0:
            print('Send to {} message \"{}\".'.format( patient_ID[i], msg_token.loc[i,"msg_tokens"]))# Sollte send Befehl werden
            Send_to_Phone(msg_token.loc[i,"msg_tokens"])